In [25]:
import numpy as np

import seaborn as sns
import pandas as pd
import polars as pl
import os
import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import cvxpy as cp
import itertools
from scipy.special import logsumexp
from scipy.sparse import csr_matrix

sns.set(style='darkgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/vivarium-ecoli'))

from ecoli.processes.metabolism_redux import NetworkFlowModel, FlowResult

In [26]:
# Reading the matrix CSV files
def read_matrix(file_path, sparse=False):
    # add sparse matrix reading
    if sparse:
        return csr_matrix(pl.read_csv(file_path, has_header=False).to_numpy())
    else:
        return pl.read_csv(file_path, has_header=False).to_numpy()


# Import external data

In [30]:
# genes
gn = pl.read_csv("notebooks/fbagd/external_data/gene_protein.txt", separator='\t')

# reactions
rx = pl.read_csv("notebooks/fbagd/external_data/reactions.txt", separator='\t')

# ribosome profiling data
li = pl.read_csv("notebooks/fbagd/external_data/Li_2014_profiling.csv")

# C matrix as np sparse matrix
C = read_matrix("notebooks/fbagd/external_data/C_matrix.csv", sparse=True)

# join ribosome profiling data with genes on gene id
li = li.join(gn, left_on='Gene', right_on='Common-Name')
li.head()

Gene,MOPS complete,MOPS minimal,MOPS complete without methionine,Gene Name,Accession-1,Product
str,i64,i64,i64,str,str,str
"""menF""",165,67,70,"""EG12362""","""b2265""","""MENF-MONOMER"""
"""mukE""",995,263,806,"""EG11252""","""b0923""","""EG11252-MONOME…"
"""psiF""",17,454,10,"""EG11401""","""b0384""","""EG11401-MONOME…"
"""hisL""",913,640,170,"""EG11269""","""b2018""","""EG11269-MONOME…"
"""yjbR""",532,541,481,"""EG11936""","""b4057""","""EG11936-MONOME…"


In [31]:
rx.head()

Reaction,EC-Number,Enzymes of a reaction
str,str,str
"""RXN-16650""","""EC-2.4.99.28""","""CPLX0-7717 // …"
"""RXN-5822""","""EC-3.1.3.108""","""ALKAPHOSPHA-CP…"
"""RXN-17355""",null,"""CPLX0-7938"""
"""RXN-15356""",null,null
"""RXN-22735""","""EC-6.3.4.16""","""CARBPSYN-CPLX"""


# Get stoichiometric matrix

In [32]:
time = '10'
date = '2024-04-27'
experiment = 'convex_kinetics'
entry = f'{experiment}_{time}_{date}'
folder = f'out/cofactors/{entry}/'

In [33]:
output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

In [34]:
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

In [60]:
stoichiometry = agent['ecoli-metabolism-redux-classic'].stoichiometry
process_rxn_names = agent['ecoli-metabolism-redux-classic'].reaction_names
process_met_names = agent['ecoli-metabolism-redux-classic'].metabolite_names
process_rxn_enz_map = agent['ecoli-metabolism-redux-classic'].catalyzed_rxn_enzymes_idx
process_enz_names = agent['ecoli-metabolism-redux-classic'].catalyst_ids

# maintenance_reaction = agent['ecoli-metabolism-redux-classic'].model.maintenance_reaction
# stoichiometry["maintenance_reaction"] = maintenance_reaction

bad_rxns = ["RXN-12440", "TRANS-RXN-121", "TRANS-RXN-300", "TRANS-RXN-8", "R15-RXN-MET/CPD-479//CPD-479/MET.25.","DISULFOXRED-RXN[CCO-PERI-BAC]-MONOMER0-4152/MONOMER0-4438//MONOMER0-4438/MONOMER0-4152.71."]
# generate carbon mistake in parca, efflux/influx proton gen, iron cheating, mass gen
# for rxn in bad_rxns:
#    stoichiometry.pop(rxn, None)


# Load enzyme counts

In [66]:
# from li, pick out mops minimal and product columns
 = li.select(["Product", "MOPS minimal"])

In [65]:
li

Gene,MOPS complete,MOPS minimal,MOPS complete without methionine,Gene Name,Accession-1,Product
str,i64,i64,i64,str,str,str
"""menF""",165,67,70,"""EG12362""","""b2265""","""MENF-MONOMER"""
"""mukE""",995,263,806,"""EG11252""","""b0923""","""EG11252-MONOME…"
"""psiF""",17,454,10,"""EG11401""","""b0384""","""EG11401-MONOME…"
"""hisL""",913,640,170,"""EG11269""","""b2018""","""EG11269-MONOME…"
"""yjbR""",532,541,481,"""EG11936""","""b4057""","""EG11936-MONOME…"
"""ybcL""",20,34,6,"""G6301""","""b0545""","""G6301-MONOMER"""
"""cynR""",152,47,94,"""EG11421""","""b0338""","""PD00291"""
"""feaB""",12,28,7,"""G7961""","""b1385""","""PHENDEHYD-MONO…"
"""ybhP""",8,127,0,"""G6407""","""b0790""","""G6407-MONOMER"""


In [ ]:
# To start with, we will be simulating MOPS minimal. 